In [8]:
from tavily import TavilyClient


In [9]:
tavily_client = TavilyClient()
results = tavily_client.search("What is the tavily api used for", max_results=2, include_raw_content=True)
results

{'query': 'What is the tavily api used for',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Introduction | Tavily AI',
   'url': 'https://docs.tavily.com/docs/welcome',
   'content': "However, we've learned that none of today's search engines provide a suitable tool that provides factual, explicit and objective answers without the need to continuously click and explore multiple sites for a given research task.\n All in a single API call!\nTo learn how to build your AI application with Tavily Search API, check out the documentation page.\n Scraping doesn't scale and requires expertise to refine, current search engine APIs don't provide explicit information to queries but simply potential related articles (which are not always related), and are not very customziable for AI agent needs. GPT Researcher is an autonomous agent that takes care of the tedious task of research for you, by scraping, filtering and aggregating over 20+ web sources per a singl

In [10]:
from llama_index.core import Document, VectorStoreIndex

documents = [Document(text=result["content"]) for result in results["results"]]
len(documents)

2

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

def setup_models():
    """Setting up the embedding and language models..."""
    Settings.embed_model = OllamaEmbedding(model_name="all-minilm:latest", base_url="http://localhost:11434", ollama_additional_kwargs={"mirostat": 0}, request_timeout=3600.0)
    Settings.llm = Ollama(model="llama3.2:1b", request_timeout=60.0)

setup_models()

# pass_embedding = Settings.embed_model.get_text_embedding_batch(
#     ["This is a passage!", "This is another passage"], show_progress=True
# )
# print(pass_embedding)

# query_embedding = Settings.embed_model.get_query_embedding("Where is blue?")
# print(query_embedding)

# response = Settings.llm.complete("What is the capital of France?")
# print(response)

In [11]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [12]:
print(nodes[0].get_content(metadata_mode="all"))

However, we've learned that none of today's search engines provide a suitable tool that provides factual, explicit and objective answers without the need to continuously click and explore multiple sites for a given research task.
 All in a single API call!
To learn how to build your AI application with Tavily Search API, check out the documentation page.
 Scraping doesn't scale and requires expertise to refine, current search engine APIs don't provide explicit information to queries but simply potential related articles (which are not always related), and are not very customziable for AI agent needs. GPT Researcher is an autonomous agent that takes care of the tedious task of research for you, by scraping, filtering and aggregating over 20+ web sources per a single research task.
 This is why we're excited to introduce the first search engine for AI agents - Tavily Search API.


In [13]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [14]:
query_engine.query("What is the tavily api used for")

Response(response="The Tavily Search API is designed specifically for building AI applications, particularly autonomous agents. It's built to scale and provide exclusive data to enhance research tasks, making it an ideal solution for new creators, growing businesses, and those requiring a high level of customizability and scalability.", source_nodes=[NodeWithScore(node=TextNode(id_='9d04429d-a7c5-4883-860d-f2bf7abb4ffa', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7bda0bc3-ea79-41d3-b8e7-5f86ce80ac8b', node_type='4', metadata={}, hash='f94d1c6a772a256e339677c660152eda8a2fae68116013e6bcc064ea993048b4')}, metadata_template='{key}: {value}', metadata_separator='\n', text="However, we've learned that none of today's search engines provide a suitable tool that provides factual, explicit and objective answers without the need to continuously click and explore multiple site

In [ ]:
from llama_index.core.tools import FunctionTool
from typing import List


def vector_query(
    query: str, 
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    """

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [15]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary a document or documents."
    ),
)

In [20]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_query_tool, summary_tool], 
    llm=Settings.llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [23]:
response = agent.chat("What is tavily used for?")
print(response)

Added user message to memory: What is tavily used for?
=== Calling Function ===
Calling function: vector_tool with args: {"query": "what is tavily used for"}
=== Function Output ===
Tavily is used as a platform to power AI research assistants, allowing new creators building businesses to focus on their goals rather than spending time researching. It provides an automated research solution that can be integrated into various applications and projects to deliver accurate and factual results at speed.
=== LLM Response ===
{}

(name: "business_solution", value: "Tavily provides a scalable and reliable platform for AI research assistants, allowing businesses to focus on their core activities while leveraging Tavily's automated research capabilities.")
{}

(name: "business_solution", value: "Tavily provides a scalable and reliable platform for AI research assistants, allowing businesses to focus on their core activities while leveraging Tavily's automated research capabilities.")
